# [C4ML - 2019](https://web.archive.org/web/20190218200248/https://www.c4ml.org/)

I attended the innagural "Compilers for Machine Learning" workshop this year and want to collect my thoughts to process them and share them with others.

I will go through each talk and then present overall findings. 

## Talks
### Julia

We (the PL/compiler community) has solved the problem of how to create a language that composes nicely. This is shown in Julia by interop between libraries in different stacks, like those implementing numbers, arrays, linear algebra etc. This is done through pattern matching and promotion between types (`Base.promote_rule`)


So we can do things like build XLA backend easily and combine different ML tools.

#### Multiple Dispatch
[Julia - "Methods"](https://docs.julialang.org/en/v1/manual/methods/)

> Julia allows the dispatch process to choose which of a function's methods to call based on the number of arguments given, and on the types of all of the function's arguments. This is different than traditional object-oriented languages, where dispatch occurs based only on the first argument, which often has a special argument syntax, and is sometimes implied rather than explicitly written as an argument.


#### Promotion
[Julia - "Conversion and Promotion"](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/index.html)
> In a sense, Julia falls into the "no automatic promotion" category: mathematical operators are just functions with special syntax, and the arguments of functions are never automatically converted. However, one may observe that applying mathematical operations to a wide variety of mixed argument types is just an extreme case of polymorphic multiple dispatch – something which Julia's dispatch and type systems are particularly well-suited to handle. "Automatic" promotion of mathematical operands simply emerges as a special application: Julia comes with pre-defined catch-all dispatch rules for mathematical operators, invoked when no specific implementation exists for some combination of operand types. These catch-all rules first promote all operands to a common type using user-definable promotion rules, and then invoke a specialized implementation of the operator in question for the resulting values, now of the same type. User-defined types can easily participate in this promotion system by defining methods for conversion to and from other types, and providing a handful of promotion rules defining what types they should promote to when mixed with other types.

Julia has two primary concepts here, conversion and promotion:

> The convert function generally takes two arguments: the first is a type object and the second is a value to convert to that type. The returned value is the value converted to an instance of given type.

> Promotion to a common "greater" type is performed in Julia by the promote function, which takes any number of arguments, and returns a tuple of the same number of values, converted to a common type, or throws an exception if promotion is not possible. The most common use case for promotion is to convert numeric arguments to a common type:

The way I think about it is that conversion is used for object creation, and promotion is used in method defintions. i.e. if you try to create an object it will promote it automatically


> Although one could, in principle, define methods for the promote function directly, this would require many redundant definitions for all possible permutations of argument types. Instead, the behavior of promote is defined in terms of an auxiliary function called promote_rule, which one can provide methods for. The promote_rule function takes a pair of type objects and returns another type object, such that instances of the argument types will be promoted to the returned type.

You define functions mapping two types to their promoted type. 

### Tiark Rompf

He has built a system for DSLs in Scala, called ["Lightweight Modular Staging" (LMS)](http://scala-lms.github.io/).

General idea is that compiler becomes bottleneck between programmer and hardware, so we fix this by adding leveled compilers (Tensor/SQL layer => Array/loop layer => SIMD, etc). 

It's like Tensorflow but prettier, i.e. integrated better into the language. Uses `Rep[Integer]` to be the type of an `Integer` that is not yet computed. But it all works nicely at language level. 

It failed partially because *it wasn't in Python* so no one cared.

So he made a version for python called [`snek-lms`](https://github.com/jmd1011/snek-LMS#lightweight-syntax-for-building-computation-graphs). It analyzes AST to turn control flow into graph.

This:

```python
@lms
def loop(n):
    x = 0
    while x < n:
        x = x + 1
    return x
```

Turns into this:

```python
def loop(n):
    x = __new_var()
    __assign(x, 0)

    def cond$1():
        return (__read(x) < n)

    def body$1():
        __assign(x, (__read(x) + 1))

    __while(cond$1, body$1)

    return __read(x)
```

Which creates this IR:


```python
[['let', x5, ['new_var']],
 ['let', x6, ['assign', x5, 0]],
 ['let', x7, ['while',
    [['let', x7, ['read', x5]],
     ['let', x8, ['<', x7, in]],
     x8],
    [['let', x7, ['read', x5]],
     ['let', x8, ['+', x7, 1]],
     ['let', x9, ['assign', x5, x8]],
     None]]],
 ['let', x8, ['read', x5]], x8]
```


He is working witth Google on this for their [Tensorflow AutoGraph](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/autograph) project.

He said that `snek-lms` is not TF or XLA specific and that Google folks working on autograph would likely be willing to accomdate non TF use cases as well. That the code is factored so it could be used without TF. 

---


### TVM - Tianqi Chen
TVM is OS ML stack out of Univ. of Washington.

He has a colleague who developed a new ASIC and he wanted to collaborate to see how it performed. He didn't know where to plug this into the current ML stacks to compare it against other hardware.

He had experience in MX Net where the idea is to have a high level computation graph, with expression as nodes. This abstractions fails for two reasons:

1. Then each high level node needs mapping to low level. What if hardware doesn't support that operation?
2. Operator fusion is essential for performance. But hand coding fusion is too time consuming/exponential number of combinations. 

So instead -> "Learning based learning systems"

1. Start with tensor expression language
2. Compute possible compile varients
3. Use statistical cost model to figure out best. This involves actually testing different versions and gathering data.

Compilation about an hour when we don't have existing statistics. 

### Glow - Facebook's IR

Practical, simple, hardware backed IR. Only static shapes, no control flow, no branching. Unroll all loops before.

ONNIXIFI is the bridge between PyTorch and Glow.  

It supports many different proprietary hardware. All that is closed source. Used internally at FB to target this hardware from PyTorch. It also support backend specific instructions and partioning based of large graph into multiple nodes based on memory constraints.

On the CPU it transforms to LLVM by using `libjit` a bunch of C++ operations on tensorors. For example this is [the tranpose](https://github.com/pytorch/glow/blob/312ce2113117bec464b4db84bc1dd2c6c2d7d982/lib/Backends/CPU/libjit/libjit.cpp#L435-L515):


```c++
void libjit_transpose_generic(const T *inW, T *outW, const size_t *idim,
                              const size_t *odim, const size_t *shuffle,
                              size_t numDims) {
  // Transpose 2d matrices one tile at a time. This access pattern ensures
  // that the whole tile is kept in L1 cache. When scanning the whole row at
  // once we invalidate many cache lines when we touch a single column.
  const unsigned tileSize = 64;

  // Source coordinate.
  size_t SC[5];

  if (numDims == 5) {
    for (size_t x = 0; x < odim[0]; x++)
      for (size_t y = 0; y < odim[1]; y++)
        for (size_t z = 0; z < odim[2]; z++)
          for (size_t w = 0; w < odim[3]; w++)
            for (size_t q = 0; q < odim[4]; q++) {
              SC[shuffle[0]] = x;
              SC[shuffle[1]] = y;
              SC[shuffle[2]] = z;
              SC[shuffle[3]] = w;
              SC[shuffle[4]] = q;
              outW[libjit_getXYZWQ(odim, x, y, z, w, q)] =
                  inW[libjit_getXYZWQ(idim, SC[0], SC[1], SC[2], SC[3], SC[4])];
            }
    return;
  }
  ...
```


### XLA

Why use XLA? Well it's the only way to run your code on a TPU.

Lot's of thought goes into fusion, when to fuse and heauristics around this. It's a black box system though, you don't have any influence over how it fuses. Also has to figure out tiling and unrolling. 


"Stencil" is the intesting abstraction here. They break out all operations into:

* memory access
* computation

and split these parts apart. So it becomes easier to compose computations and do fusion.

However, all operations are hardcoded in core. They can only be added by modifying core.

### Intel - nGraph
It is an IR, a runtime, and a compiler. It does it all! 

### TACO

Fusion is more important for sparse operations.

The input is an enisum lik eoperation.

Sparse arrays are implemented in different forms. Each "level" or dimension of the array can have a different representation like:

* Dense
* Hashed
* ...

Each of these represenations have different properties.

Primary is difference between indexed vs. iterable layers. Some layers have O(1) indexing, others have O(1) iteration, which require different code to be generated.

More details are in ["Format Abstraction for Sparse Tensor Algebra Compilers"](http://tensor-compiler.org/chou-oopsla18-taco-formats.pdf) paper.


It also has a policy language specify which path to take, with different types of heuristics.

### PlaidML

By Intel, used on top of ngraph, but also goes to other backends.

Uses polyhedral compilation and the primary abstraction is a "block". Block nesting is the same as dimension lifitng. Also similar to Dask's abstraction level. 

The right granularity for these IRs is graphs of reordable loops "loop nest". 

### Nvidia - Diesel

We need new compiler for parallel graphs and parallel hardware. 

They are "doing what everyone is doing", which is JITing once they know dimensions. 

Use a "l

## Conversations

### Chris Lattner
TVM has great ideas but is active research project and isn't stable. Compile times are too long.

All systems demonstrated here are good but they are usually serving short term business goals. So they don't design for the long hall. That's what he is trying to do.

Both Tensorflow and TVM are also showing their warts. XLA is limited to static shapes. Tensorflow has many different frontends and backends. It's a mess internally trying to implement conversions between them. 


## Implications 



### ML in Python

Python is **the** target for all of these tools. However they often view it as a sort of last-part dumping ground. It seems that providing Pythonic APIs is not top priority or thinking about how Python user's interact with their tool. 

---

Julia's take is that it's nice to write ML all inside one language, without switching between them. Allows sharing of tools and movement up and down the stack more reasily.

What is Python's "take" here?

To put a positive spin on it, we could say that in Julia, it tries to *own* the computation space. As in, it wants control and ownership over what's going on. If we take the opposite position, that would be very unopinionated about where/who does the computataion (fortran compiled function, tensorflow C++ executor, AOT compilation), but just let you stitch those different systems together and offer helpful abstractions over them.

---

After Tiark Rompf's talk I asked about integrating LMS with Python more directly and how this work could impact the broader community. He has no interest of attending PyCon or interacting with Python community to talk about language level changes to support this kind of thing.

His advice was to get google people working on autograph to do this. Maybe by creating a ML for Python group and having people from different industry as a part of it to inform language development.



### `uarray`

From Julia, both conversion and promotion are useful concepts to copy hopefully.
  * Conversion: This would be useful to get remove some of the noisy casting between things like "array" and "MoA" etc.
  * Promotion: It's possible we could use this for AST creation. For example, if we have `Natural(a) + Natural(b)` if a is an integer and b is an AST node the integer should be promoted to an ast node and then the conversion would happen. So this would be instead of use (problematic) `to_ast` function which is hard to understand when/what it should convert. This reframes the question by removing that function, and instead rely on conversion semantics to think about compilation.
  
---


We could use `snek-lms` instead of creating lazy ndarray to create graph. Two reasons:

1. Woud allow us to support dynamic control flow/conditionals in native python syntax
2. might be easier to extract original line numbers of python code to tie them to output for debugging purposes.



---

There are crazy amounts of work to compile deep learning to lowest levels, including quantization (choosing what size weights to use), tiling (so that compute/memory access is overlapping as much as possible). We should stick to the high level, as much as possible! This means offloading as much work as we can, and staying away from MoA pipelines (or at least not neccesarily making them primary). Many pipeliens do optimizations at whole operation level (i.e. transpose of matrix multiplication) so we need to pass it to them in this form if they are to optimize it.

We should target `ngraph`, `PlaidML`, Taco, XLA, TVM.

---

To properly target sparse matrices we need to adopt TACO mentality. This means talking about arrays as either allowing direct indexing OR iterationg, by dimension. This is fundamentally different than thinking about them exclusively as O(1) mapping from indices to values. 

"Highest level" representation is really this iterable idea. i.e. we can build performant iteration on top of indexing, but not vice versa. So iteration, by dimension, is "lowest"  type of tensor.

We need to figure out how to integrate this at API level. 